In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install flask-ngrok

In [4]:
!mkdir images

!mkdir Yolo-images

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
# install dependencies as necessary
!pip install -r yolov5/requirements.txt  # install dependencies

%cd yolov5

In [6]:

import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.8.1+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [7]:
import os
from flask_ngrok import run_with_ngrok
from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename

UPLOAD_FOLDER = '/content/images'
BLURRED_FOLDER = '/content/Yolo-images'

ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'mp4','avi','jpeg', 'gif'}

app = Flask(__name__, static_folder='static')
app.config['SESSION_TYPE'] = 'memcached'
app.config['SECRET_KEY'] = 'super secret key'
run_with_ngrok(app)   #starts ngrok when the app is run

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
expIndex = 0

In [8]:
import cv2
import numpy as np
import time
import sys
from google.colab.patches import cv2_imshow


@app.route('/upload/yolo', methods=['GET', 'POST'])
def upload_file_alphas():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            path_l = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            global expIndex
            expIndex = expIndex+1
            !python detect.py --weights /content/drive/MyDrive/ObjectDetection/yoloV5l/best.pt --img 416 --conf 0.4 --source $path_l 
            return redirect(url_for('yolo_image',
                                    filename=filename))
        
    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload>
    </form>
    '''

In [9]:
@app.route("/")
def home():
    return "<h1>Running Rehabsoft Vision on Google Colab!</h1>"


def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS





In [10]:
from flask import send_from_directory



@app.route('/yolo/result//<filename>')
def yolo_image(filename):
    global expIndex  
    if(expIndex<2):
        output_path = "/content/yolov5/runs/detect/exp" 
    else:
        output_path = "/content/yolov5/runs/detect/exp" + str(expIndex) 
    print(output_path)
    return send_from_directory(output_path, filename)

In [11]:

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9c516d609390.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [11/Jun/2021 22:55:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2021 22:55:53] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2021 22:56:00] "GET /upload/yolo HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2021 22:56:01] "GET /favicon.ico HTTP/1.1" 404 -


Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', exist_ok=False, half=False, hide_conf=False, hide_labels=False, imgsz=416, iou_thres=0.45, line_thickness=3, max_det=1000, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/images/zassd.jpg', update=False, view_img=False, weights=['/content/drive/MyDrive/ObjectDetection/yoloV5l/best.pt'])
YOLOv5 🚀 v5.0-181-g4984cf5 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 308 layers, 21154827 parameters, 0 gradients
image 1/1 /content/images/zassd.jpg: 416x416 3 Bell_pepper leafs, Done. (0.081s)
Results saved to runs/detect/exp
Done. (0.149s)


...

/content/yolov5/runs/detect/exp2


127.0.0.1 - - [11/Jun/2021 22:56:47] "GET /upload/yolo HTTP/1.1" 200 -


Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', exist_ok=False, half=False, hide_conf=False, hide_labels=False, imgsz=416, iou_thres=0.45, line_thickness=3, max_det=1000, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/images/leaves-of-raspberry-stock-picture-2529053_jpg.rf.a90e5533d45aa55cd41512ff4c5b9659.jpg', update=False, view_img=False, weights=['/content/drive/MyDrive/ObjectDetection/yoloV5l/best.pt'])
YOLOv5 🚀 v5.0-181-g4984cf5 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 308 layers, 21154827 parameters, 0 gradients
image 1/1 /content/images/leaves-of-raspberry-stock-picture-2529053_jpg.rf.a90e5533d45aa55cd41512ff4c5b9659.jpg: 416x416 3 Raspberry leafs, Done. (0.021s)
Results saved to runs/detect/exp3
Done. (0.030s)


127.0.0.1 - - [11/Jun/2021 22:57:06] "POST /upload/yolo HTTP/1.1" 302 -
127.0.0.1 - - [11/Jun/2021 22:57:06] "GET /yolo/result/leaves-of-raspberry-stock-picture-2529053_jpg.rf.a90e5533d45aa55cd41512ff4c5b9659.jpg HTTP/1.1" 200 -


/content/yolov5/runs/detect/exp3
